## Sets:

Let $I$ be the set of products (finished goods) and $T$ the set of time periods (e.g. planning horizon). Optionally, if the warehouse has multiple floors or zones, include a set $F$ of floors.

## Parameters:

Define parameters such as demand $d_{i,t}$ (units of item $i$ required in period $t$), production capacity $P_t$ in period $t$, storage space (e.g. total warehouse capacity) $W$, and the space usage $a_i$ per unit of item $i$. Also include cost parameters: production cost $c^p_{i,t}$ (per unit), fixed production/setup cost $F_{i,t}$ (if any), and inventory holding cost $h_{i,t}$ per unit carried over. For example, $W$ is the total available storage space for finished goods.

## Decision Variables:

Introduce variables for production and inventory. For example, let $x_{i,t}\ge 0$ be the production quantity of item $i$ in period $t$, and $I_{i,t}\ge 0$ the ending inventory of item $i$ at period $t$. If modeling setup costs, include a binary $y_{i,t}\in{0,1}$ that is 1 if production of item $i$ occurs in period $t$. (If multiple floors are modeled explicitly, one might include $z_{i,f,t}$ inventory of $i$ on floor $f$, but under a randomized policy a single inventory variable per item is sufficient since any item can occupy any space.)


## The objective function

to minimize the total cost of production and warehousing.
In Mathematical form:

### $Σ_(t∈T) Σ_(i∈I) (c^p_{i,t}*x_{i,t} + F_{i,t}*y_{i,t} + h_{i,t}*I_{i,t})$

This sums production costs $c^p{i,t}*x{i,t}$, any fixed (setup) costs $F_{i,t}*y_{i,t}$, and holding costs $h_{i,t}*I_{i,t}$ over all items and periods. This follows the formulation in Zhang et al., who model the strategy as an ILP minimizing total production and warehouse cost.


### Constraints:

1. Inventory Balance: For each item $i$ and period $t$, ensure that inventory flows account for production, demand, and previous inventory:
   $I_{i,t-1} + x_{i,t} = d_{i,t} + I_{i,t}$, where $I_{i,0} = 0$.
   This constraint forces supply plus carry-over to meet demand.

2. Production Capacity: In each period $t$, the total production cannot exceed available capacity:
   $Σ_(t∈T) x_{i,t}<=P_{t}$
   This ensures the model respects machine/time limits.

3. Setup Linking (if used): If setup decisions are modeled, link production and setup via a big‑M constraint, e.g.:
   $x_{i,t} <= M*y_{i,t} (for all i,t)
where M is a large constant (e.g. $M=P_t$). This forces $y_{i,t}=1$ whenever $x_{i,t}>0$

4. Inventory Bounds: If each item has a storage limit, enforce $I_{i,t} \le I^{\max}i$. In many cases the only limit is total space, so individual $I{i,t}$ may be unbounded except by demand.

5. Warehouse Storage Capacity: Under the randomized policy, each unit of inventory occupies space $a_i$, and only the total used space matters (since any item can go to any location). Enforce in each period $t$:
   $Σ_(t∈T) a_{i}*I_{i,t}<=W$
   This ensures total stored volume does not exceed warehouse capacity. (Because of IoT-enabled random storage, we do not assign items to specific slots; instead the aggregated capacity constraint suffices.)

6. Nonnegativity and Integrality: All $x_{i,t},I_{i,t}\ge0$, and $y_{i,t}\in{0,1}$ if used.


#### Importing Libraries


In [50]:
from gurobipy import *
import gurobipy as gp

In [51]:
Model = gp.Model("Prob")

In [52]:
items = [f'item{i}' for i in range(6)] # Assuming there are 10 items
T = 5  # Assuming time period T=5
periods = range(1, T+1) 

In [53]:
# Production cost
c_prod = {(i, t): 10 + (idx + t) % 5 for idx, i in enumerate(items) for t in periods}

# Setup cost
F_setup = {(i, t): 100 + (idx + t) * 10 for idx, i in enumerate(items) for t in periods}

# Holding cost
h_cost = {(i, t): 2 + (idx + t) % 3 for idx, i in enumerate(items) for t in periods}

demand = {(i, t): 20 + (idx + t) * 2 for idx, i in enumerate(items) for t in periods}
max_prod = {(i, t): 100 for i in items for t in periods}  # Max production capacity
warehouse_capacity = {t: 400 for t in periods}  # Max Warehouse space per period

In [64]:
c_prod

{('item0', 1): 11,
 ('item0', 2): 12,
 ('item0', 3): 13,
 ('item0', 4): 14,
 ('item0', 5): 10,
 ('item1', 1): 12,
 ('item1', 2): 13,
 ('item1', 3): 14,
 ('item1', 4): 10,
 ('item1', 5): 11,
 ('item2', 1): 13,
 ('item2', 2): 14,
 ('item2', 3): 10,
 ('item2', 4): 11,
 ('item2', 5): 12,
 ('item3', 1): 14,
 ('item3', 2): 10,
 ('item3', 3): 11,
 ('item3', 4): 12,
 ('item3', 5): 13,
 ('item4', 1): 10,
 ('item4', 2): 11,
 ('item4', 3): 12,
 ('item4', 4): 13,
 ('item4', 5): 14,
 ('item5', 1): 11,
 ('item5', 2): 12,
 ('item5', 3): 13,
 ('item5', 4): 14,
 ('item5', 5): 10}

#### Making variables


In [55]:
x = Model.addVars(items, periods, name='x', lb=0)
I = Model.addVars(items, periods, name='I', lb=0)
y = Model.addVars(items, periods, name='y', vtype=GRB.BINARY)

#### Setting the objective function


In [56]:
Model.setObjective(
    gp.quicksum(c_prod[i,t]*x[i,t] + F_setup[i,t]*y[i,t] + h_cost[i,t]*I[i,t]
    for i in items for t in periods),
    GRB.MAXIMIZE
)

#### Inventory balance: I[i,t] = I[i,t-1] + x[i,t] - demand[i,t]


In [57]:
for i in items:
    for t in periods:
        if t == 1:
            Model.addConstr(I[i, t] == x[i, t] - demand[i, t], name=f"inv_balance_{i}_{t}")
        else:
            Model.addConstr(I[i, t] == I[i, t-1] + x[i, t] - demand[i, t], name=f"inv_balance_{i}_{t}")

#### Production only if setup is done


In [58]:
for i in items:
    for t in periods:
        Model.addConstr(x[i, t] <= max_prod[i, t] * y[i, t], name=f"setup_limit_{i}_{t}")

#### Warehouse capacity constraint (total inventory across items ≤ capacity)


In [59]:
for t in periods:
    Model.addConstr(
        gp.quicksum(I[i, t] for i in items) <= warehouse_capacity[t],
        name=f"warehouse_capacity_{t}"
    )

#### Initial inventory = 0


In [60]:
for i in items:
    Model.addConstr(I[i, 1] >= 0, name=f"init_inventory_{i}")

In [61]:
Model.write('Warehouse management.lp')

#### Model optimization


In [62]:
Model.optimize()

Gurobi Optimizer version 12.0.2 build v12.0.2rc0 (win64 - Windows 11.0 (26100.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-13700HX, instruction set [SSE2|AVX|AVX2]
Thread count: 16 physical cores, 24 logical processors, using up to 24 threads

Optimize a model with 71 rows, 90 columns and 180 nonzeros
Model fingerprint: 0xc81e8634
Variable types: 60 continuous, 30 integer (30 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+02]
  Objective range  [2e+00, 2e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [2e+01, 4e+02]
Found heuristic solution: objective 15810.000000
Presolve removed 42 rows and 36 columns
Presolve time: 0.00s
Presolved: 29 rows, 54 columns, 102 nonzeros
Variable types: 54 continuous, 0 integer (0 binary)

Root relaxation: objective 2.786800e+04, 33 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    

#### Output Solution


In [63]:
if Model.status == GRB.OPTIMAL:
    print("Optimal Total Cost:", Model.objVal)
    for i in items:
        for t in periods:
            print(f"{i}, Period {t}: x={x[i, t].X}, y={y[i, t].X}, I={I[i, t].X}")


Optimal Total Cost: 27868.0
item0, Period 1: x=100.0, y=1.0, I=78.0
item0, Period 2: x=74.0, y=1.0, I=128.0
item0, Period 3: x=10.0, y=1.0, I=112.0
item0, Period 4: x=100.0, y=1.0, I=184.0
item0, Period 5: x=0.0, y=1.0, I=154.0
item1, Period 1: x=100.0, y=1.0, I=76.0
item1, Period 2: x=0.0, y=1.0, I=50.0
item1, Period 3: x=100.0, y=1.0, I=122.0
item1, Period 4: x=0.0, y=1.0, I=92.0
item1, Period 5: x=0.0, y=1.0, I=60.0
item2, Period 1: x=100.0, y=1.0, I=74.0
item2, Period 2: x=100.0, y=1.0, I=146.0
item2, Period 3: x=0.0, y=1.0, I=116.0
item2, Period 4: x=0.0, y=1.0, I=84.0
item2, Period 5: x=10.0, y=1.0, I=60.0
item3, Period 1: x=100.0, y=1.0, I=72.0
item3, Period 2: x=0.0, y=1.0, I=42.0
item3, Period 3: x=0.0, y=1.0, I=10.0
item3, Period 4: x=24.0, y=1.0, I=0.0
item3, Period 5: x=100.0, y=1.0, I=64.0
item4, Period 1: x=96.0, y=1.0, I=66.0
item4, Period 2: x=0.0, y=1.0, I=34.0
item4, Period 3: x=0.0, y=1.0, I=0.0
item4, Period 4: x=36.0, y=1.0, I=0.0
item4, Period 5: x=100.0, y=1.0, I